# Code for VBM analysis
- Code adapted from: [Grégory Operto](http://xgrg.github.io/VBM-voxelwise-multiple-regression-in-Python/)
- and niLearn example: [niLearn VBM example](https://nilearn.github.io/auto_examples/02_decoding/plot_oasis_vbm.html)
- Uses subject list from:
- Uses cerebellar brain-masks from freesurfer or mni template 
## Cohorts
- Essential Tremor, Normal Controls
## Tasks
- VBM for MNI ET/NC and MNI ET/{MERGED_NC}
## Covariates
- Age, Sex, site, TBV

In [1]:
import pandas as pd
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import nibabel as nib

sys.path.append('../')
from lib.stats_utils import *


Bad key "text.kerning_factor" on line 4 in
/usr/local/miniconda/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


### Paths

In [16]:
project_dir = '../../../'
data_dir = '/codes/tab_data/'
metadata_dir = '../metadata/'
results_dir = '/codes/tab_data/MAGeT/'

### Participant demographics

In [10]:
participant_df = pd.read_csv('{}participants.csv'.format(data_dir))

# exclusions
exclude_subjects = list(participant_df[participant_df['Include in analysis']==0]['Participant_id'])
exclude_subjects = exclude_subjects + ['sub-0123'] # Age unknow ['?']

participant_df = participant_df[~participant_df['Participant_id'].isin(exclude_subjects)]
include_subjects = participant_df['Participant_id'].values
print('Total number of participants included: {} (excluded: {})'.format(len(include_subjects),len(exclude_subjects)))

# reformat columns
participant_df['Age'] = participant_df['Age'].astype(np.int)
age_df = participant_df.groupby(['group','Sex'])[['Age']].mean()
sex_df = participant_df.groupby(['group','Sex']).count()['Participant_id']
#print(pd.merge(age_df,sex_df,on=['group','Sex']))

## Check preproc subject count

failed_subjects = set(include_subjects) - set(participant_df[participant_df['MAGeT processing']==1]['Participant_id'])
print('\nMAGet failures: {}'.format(failed_subjects))

participant_df[participant_df['Participant_id'].isin(failed_subjects)]

Total number of participants included: 113 (excluded: 22)

MAGet failures: set()


,Participant_id,Age,Sex,group,Include in analysis,conversion Problem,opteration,T1w,T2w,dwi,...,bold fmap,swi,fmriprep (ses-1) anat only,freesurfer-6.0.1,fmriprep (ses-1) anat + bold prprocessing,T1-QC,T1-template,SUIT () cerebellum segmentation,MAGeT processing,MAGet Comments


In [2]:
# nilearn vbm
%matplotlib inline

In [ ]:
# Authors: Elvis Dhomatob, <elvis.dohmatob@inria.fr>, Apr. 2014
#          Virgile Fritsch, <virgile.fritsch@inria.fr>, Apr 2014
#          Gael Varoquaux, Apr 2014
#          Andres Hoyos-Idrobo, Apr 2017
import numpy as np
import matplotlib.pyplot as plt
from nilearn import datasets
from nilearn.input_data import NiftiMasker
from nilearn.image import get_data

n_subjects = 100  # more subjects requires more memory

In [ ]:
# load data
oasis_dataset = datasets.fetch_oasis_vbm(n_subjects=n_subjects)
gray_matter_map_filenames = oasis_dataset.gray_matter_maps
age = oasis_dataset.ext_vars['age'].astype(float)

# Split data into training set and test set
from sklearn.model_selection import train_test_split
gm_imgs_train, gm_imgs_test, age_train, age_test = train_test_split(
    gray_matter_map_filenames, age, train_size=.6, random_state=0)

# print basic information on the dataset
print('First gray-matter anatomy image (3D) is located at: %s' %
      oasis_dataset.gray_matter_maps[0])  # 3D data
print('First white-matter anatomy image (3D) is located at: %s' %
      oasis_dataset.white_matter_maps[0])  # 3D data

In [ ]:
# preproc
nifti_masker = NiftiMasker(
    standardize=False,
    smoothing_fwhm=2,
    memory='nilearn_cache')  # cache options
gm_maps_masked = nifti_masker.fit_transform(gm_imgs_train)

# The features with too low between-subject variance are removed using
# :class:`sklearn.feature_selection.VarianceThreshold`.
from sklearn.feature_selection import VarianceThreshold
variance_threshold = VarianceThreshold(threshold=.01)
gm_maps_thresholded = variance_threshold.fit_transform(gm_maps_masked)
gm_maps_masked = variance_threshold.inverse_transform(gm_maps_thresholded)

# Then we convert the data back to the mask image in order to use it for
# decoding process
mask = nifti_masker.inverse_transform(variance_threshold.get_support())

In [ ]:
# Prediction pipeline with ANOVA and SVR using :class:nilearn.decoding.DecoderRegressor Object

# In nilearn we can benefit from the built-in DecoderRegressor object to
# do ANOVA with SVR instead of manually defining the whole pipeline.
# This estimator also uses Cross Validation to select best models and ensemble
# them. Furthermore, you can pass n_jobs=<some_high_value> to the
# DecoderRegressor class to take advantage of a multi-core system.
# To save time (because these are anat images with many voxels), we include
# only the 1-percent voxels most correlated with the age variable to fit. We
# also want to set mask hyperparameter to be the mask we just obtained above.

from nilearn.decoding import DecoderRegressor
decoder = DecoderRegressor(estimator='svr', mask=mask,
                           scoring='neg_mean_absolute_error',
                           screening_percentile=1,
                           n_jobs=1)
# Fit and predict with the decoder
decoder.fit(gm_imgs_train, age_train)

# Sort test data for better visualization (trend, etc.)
perm = np.argsort(age_test)[::-1]
age_test = age_test[perm]
gm_imgs_test = np.array(gm_imgs_test)[perm]
age_pred = decoder.predict(gm_imgs_test)

prediction_score = -np.mean(decoder.cv_scores_['beta'])

print("=== DECODER ===")
print("explained variance for the cross-validation: %f" % prediction_score)
print("")

In [ ]:
# vis
weight_img = decoder.coef_img_['beta']

# Create the figure
from nilearn.plotting import plot_stat_map, show
bg_filename = gray_matter_map_filenames[0]
z_slice = 0
display = plot_stat_map(weight_img, bg_img=bg_filename,
                        display_mode='z', cut_coords=[z_slice])
display.title("SVM weights")
show()

In [ ]:
# vis prediction
plt.figure(figsize=(6, 4.5))
plt.suptitle("Decoder: Mean Absolute Error %.2f years" % prediction_score)
linewidth = 3
plt.plot(age_test, label="True age", linewidth=linewidth)
plt.plot(age_pred, '--', c="g", label="Predicted age", linewidth=linewidth)
plt.ylabel("age")
plt.xlabel("subject")
plt.legend(loc="best")
plt.figure(figsize=(6, 4.5))
plt.plot(age_test - age_pred, label="True age - predicted age",
         linewidth=linewidth)
plt.xlabel("subject")
plt.legend(loc="best")

In [ ]:
#Inference with massively univariate model

print("Massively univariate model")

gm_maps_masked = NiftiMasker().fit_transform(gray_matter_map_filenames)
data = variance_threshold.fit_transform(gm_maps_masked)

# Statistical inference
from nilearn.mass_univariate import permuted_ols
neg_log_pvals, t_scores_original_data, _ = permuted_ols(
    age, data,  # + intercept as a covariate by default
    n_perm=2000,  # 1,000 in the interest of time; 10000 would be better
    verbose=1, # display progress bar
    n_jobs=1)  # can be changed to use more CPUs
signed_neg_log_pvals = neg_log_pvals * np.sign(t_scores_original_data)
signed_neg_log_pvals_unmasked = nifti_masker.inverse_transform(
    variance_threshold.inverse_transform(signed_neg_log_pvals))

# Show results
threshold = -np.log10(0.1)  # 10% corrected

fig = plt.figure(figsize=(5.5, 7.5), facecolor='k')

display = plot_stat_map(signed_neg_log_pvals_unmasked, bg_img=bg_filename,
                        threshold=threshold, cmap=plt.cm.RdBu_r,
                        display_mode='z', cut_coords=[z_slice],
                        figure=fig)
title = ('Negative $\\log_{10}$ p-values'
         '\n(Non-parametric + max-type correction)')
display.title(title, y=1.2)

n_detections = (get_data(signed_neg_log_pvals_unmasked) > threshold).sum()
print('\n%d detections' % n_detections)

show()